<a href="https://colab.research.google.com/github/ZzangJay/team_project/blob/kimjisan/221119_patients_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive  # for colab
drive.mount('/content/gdrive/')  # for colab
cloud_directory = '/content/gdrive/MyDrive/mimiciv_unzip/'
os.listdir(cloud_directory) # for colab, check cloud directory mount

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


['CHANGELOG.txt',
 'LICENSE.txt',
 'SHA256SUMS.txt',
 'index.html',
 'icu',
 'hosp',
 'drop_2_chartevents.csv']

In [117]:
antibio=pd.read_csv('/content/gdrive/MyDrive/mimiciv preprocessing_js/icu_ab_join.csv')
sepsis3=pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/sepsis3.csv')
microbiologyevents = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/microbiologyevents.csv')
extra_features = pd.read_csv('/content/gdrive/MyDrive/mimiciv preprocessing_js/extra_features2.csv')

In [119]:
hadm_id=[]
for i in extra_features['hadm_id']:
  hadm_id.append(i)

In [108]:
# stay_id=[]
# for i in sepsis3['stay_id']:
#   stay_id.append(i)

In [118]:
antibio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371251 entries, 0 to 371250
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   subject_id    371251 non-null  int64 
 1   hadm_id       371251 non-null  int64 
 2   stay_id       371251 non-null  int64 
 3   icu_intime    371251 non-null  object
 4   icu_outtime   371251 non-null  object
 5   drug          371251 non-null  object
 6   ab_starttime  371251 non-null  object
 7   ab_stoptime   371251 non-null  object
dtypes: int64(3), object(5)
memory usage: 22.7+ MB


In [133]:
antibio_ex=antibio[antibio['hadm_id'].isin(hadm_id)]

In [134]:
print(len(antibio))
print(len(antibio_ex))

371251
316044


In [135]:
antibio_ex.columns = ['subject_id', 'hadm_id', 'stay_id', 'icu_intime', 'icu_outtime', 'drug','antibiotic_time', 'ab_stoptime']

In [136]:
antibio_ex['drug'].value_counts()

vancomycin                     117609
cefepime                        39418
piperacillin-tazobactam         30094
cefazolin                       16179
metronidazole (flagyl)          13390
                                ...  
oxacillin                           1
*nf* cefuroxime                     1
nitrofurantoin macrocrystal         1
bactrim                             1
clindamycin suspension              1
Name: drug, Length: 83, dtype: int64

In [137]:
antibio_sep=pd.merge(antibio_ex, sepsis3, on=['stay_id','antibiotic_time'])

In [138]:
antibio_sep

,subject_id_x,hadm_id,stay_id,icu_intime,icu_outtime,drug,antibiotic_time,ab_stoptime,subject_id_y,culture_time,suspected_infection_time,sofa_time,sofa_score,respiration,coagulation,liver,cardiovascular,cns,renal,sepsis3
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,sulfameth/trimethoprim ss,2131-01-11 08:00:00,2131-01-20 12:00:00,10001884,2131-01-10 16:36:00,2131-01-10 16:36:00,2131-01-11 05:00:00,3,0,0,0,3,0,0,True
1,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,vancomycin,2131-01-11 08:00:00,2131-01-12 10:00:00,10001884,2131-01-10 16:36:00,2131-01-10 16:36:00,2131-01-11 05:00:00,3,0,0,0,3,0,0,True
2,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,vancomycin,2131-01-11 08:00:00,2131-01-11 09:00:00,10001884,2131-01-10 16:36:00,2131-01-10 16:36:00,2131-01-11 05:00:00,3,0,0,0,3,0,0,True
3,10002013,23581541,39060235,2160-05-18 10:00:53,2160-05-19 17:33:33,cefazolin,2160-05-18 11:00:00,2160-05-19 17:00:00,10002013,2160-05-18 14:40:00,2160-05-18 11:00:00,2160-05-18 15:00:00,2,1,0,0,1,0,0,True
4,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,azithromycin,2129-08-05 14:00:00,2129-08-10 13:00:00,10002155,2129-08-04 17:04:00,2129-08-04 17:04:00,2129-08-05 06:00:00,2,0,0,0,1,0,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42211,19998878,26489544,34403689,2132-09-30 21:55:00,2132-10-01 15:43:11,ciprofloxacin iv,2132-10-01 00:00:00,2132-10-01 11:00:00,19998878,2132-10-01 00:38:00,2132-10-01 00:00:00,2132-10-01 00:00:00,3,0,2,0,0,0,1,True
42212,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,ciprofloxacin iv,2162-08-19 16:00:00,2162-08-22 10:00:00,19999297,2162-08-16 00:00:00,2162-08-16 00:00:00,2162-08-16 06:00:00,7,0,3,3,0,1,0,True
42213,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,cefazolin,2148-11-19 18:00:00,2148-11-20 17:00:00,19999442,2148-11-19 17:00:00,2148-11-19 17:00:00,2148-11-19 17:00:00,3,0,1,0,0,2,0,True
42214,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,levofloxacin,2139-10-10 23:00:00,2139-10-11 09:00:00,19999625,2139-10-10 16:45:00,2139-10-10 16:45:00,2139-10-10 22:00:00,2,0,0,0,1,1,0,True


In [139]:
antibio_sep.sort_values(by=['hadm_id','antibiotic_time'])

,subject_id_x,hadm_id,stay_id,icu_intime,icu_outtime,drug,antibiotic_time,ab_stoptime,subject_id_y,culture_time,suspected_infection_time,sofa_time,sofa_score,respiration,coagulation,liver,cardiovascular,cns,renal,sepsis3
21044,14990224,20000147,30503572,2121-08-30 16:33:54,2121-08-31 21:29:49,cefazolin,2121-08-30 20:00:00,2121-09-01 12:00:00,14990224,2121-08-31 12:41:00,2121-08-30 20:00:00,2121-08-30 23:00:00,3,3,0,0,0,0,0,True
25480,16003661,20001305,36916968,2178-03-25 02:59:09,2178-03-27 21:46:10,cefepime,2178-03-25 04:00:00,2178-03-25 08:00:00,16003661,2178-03-25 02:25:00,2178-03-25 02:25:00,2178-03-25 06:00:00,2,0,1,0,1,0,0,True
19429,14577567,20001361,33475095,2143-05-04 16:52:00,2143-05-10 17:59:10,levofloxacin,2143-05-04 21:00:00,2143-05-06 10:00:00,14577567,2143-05-04 10:50:00,2143-05-04 10:50:00,2143-05-04 18:00:00,5,0,0,0,3,0,2,True
19430,14577567,20001361,33475095,2143-05-04 16:52:00,2143-05-10 17:59:10,piperacillin-tazobactam,2143-05-04 21:00:00,2143-05-05 09:00:00,14577567,2143-05-04 10:50:00,2143-05-04 10:50:00,2143-05-04 18:00:00,5,0,0,0,3,0,2,True
30104,17112572,20001687,37766114,2196-06-13 09:18:36,2196-06-15 10:34:07,cefazolin,2196-06-13 13:00:00,2196-06-15 06:00:00,17112572,2196-06-12 18:30:00,2196-06-12 18:30:00,2196-06-13 16:00:00,2,0,1,0,0,0,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21586,15129243,29999098,33475199,2128-06-10 01:18:00,2128-06-21 03:01:13,piperacillin-tazobactam,2128-06-11 19:00:00,2128-06-12 10:00:00,15129243,2128-06-10 02:02:00,2128-06-10 02:02:00,2128-06-10 02:00:00,2,0,0,0,0,2,0,True
21587,15129243,29999098,33475199,2128-06-10 01:18:00,2128-06-21 03:01:13,vancomycin,2128-06-11 19:00:00,2128-06-12 18:00:00,15129243,2128-06-10 02:02:00,2128-06-10 02:02:00,2128-06-10 02:00:00,2,0,0,0,0,2,0,True
14678,13478841,29999498,33874605,2188-05-13 12:03:00,2188-05-14 16:56:15,cefepime,2188-05-13 15:00:00,2188-05-14 14:00:00,13478841,2188-05-13 07:50:00,2188-05-13 07:50:00,2188-05-13 14:00:00,2,0,0,0,0,1,1,True
1458,10355856,29999625,36975675,2157-11-07 11:51:00,2157-11-29 15:00:36,cefazolin,2157-11-07 14:00:00,2157-11-08 03:00:00,10355856,2157-11-08 11:09:00,2157-11-07 14:00:00,2157-11-07 14:00:00,2,1,0,0,0,0,1,True


In [140]:
antibio_sep_col=antibio_sep.drop_duplicates(subset=['hadm_id', 'stay_id', 'drug'], keep='first', inplace=False, ignore_index=False)

In [141]:
len(antibio_sep_col)

39630

In [142]:
antibio_sep_col

,subject_id_x,hadm_id,stay_id,icu_intime,icu_outtime,drug,antibiotic_time,ab_stoptime,subject_id_y,culture_time,suspected_infection_time,sofa_time,sofa_score,respiration,coagulation,liver,cardiovascular,cns,renal,sepsis3
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,sulfameth/trimethoprim ss,2131-01-11 08:00:00,2131-01-20 12:00:00,10001884,2131-01-10 16:36:00,2131-01-10 16:36:00,2131-01-11 05:00:00,3,0,0,0,3,0,0,True
1,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,vancomycin,2131-01-11 08:00:00,2131-01-12 10:00:00,10001884,2131-01-10 16:36:00,2131-01-10 16:36:00,2131-01-11 05:00:00,3,0,0,0,3,0,0,True
3,10002013,23581541,39060235,2160-05-18 10:00:53,2160-05-19 17:33:33,cefazolin,2160-05-18 11:00:00,2160-05-19 17:00:00,10002013,2160-05-18 14:40:00,2160-05-18 11:00:00,2160-05-18 15:00:00,2,1,0,0,1,0,0,True
4,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,azithromycin,2129-08-05 14:00:00,2129-08-10 13:00:00,10002155,2129-08-04 17:04:00,2129-08-04 17:04:00,2129-08-05 06:00:00,2,0,0,0,1,0,1,True
5,10002155,28994087,31090461,2130-09-24 00:50:00,2130-09-27 22:13:41,levofloxacin,2130-09-24 10:00:00,2130-09-27 12:00:00,10002155,2130-09-23 20:30:00,2130-09-23 20:30:00,2130-09-24 04:00:00,3,0,0,0,1,0,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42211,19998878,26489544,34403689,2132-09-30 21:55:00,2132-10-01 15:43:11,ciprofloxacin iv,2132-10-01 00:00:00,2132-10-01 11:00:00,19998878,2132-10-01 00:38:00,2132-10-01 00:00:00,2132-10-01 00:00:00,3,0,2,0,0,0,1,True
42212,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,ciprofloxacin iv,2162-08-19 16:00:00,2162-08-22 10:00:00,19999297,2162-08-16 00:00:00,2162-08-16 00:00:00,2162-08-16 06:00:00,7,0,3,3,0,1,0,True
42213,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,cefazolin,2148-11-19 18:00:00,2148-11-20 17:00:00,19999442,2148-11-19 17:00:00,2148-11-19 17:00:00,2148-11-19 17:00:00,3,0,1,0,0,2,0,True
42214,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,levofloxacin,2139-10-10 23:00:00,2139-10-11 09:00:00,19999625,2139-10-10 16:45:00,2139-10-10 16:45:00,2139-10-10 22:00:00,2,0,0,0,1,1,0,True


In [143]:
ab_class=pd.read_csv('/content/gdrive/MyDrive/mimiciv preprocessing_js/antibiotics_Class.csv')

In [144]:
for i in tqdm(antibio_sep_col['drug'],desc='convert',total=39630):
  for j in range(len(ab_class['Antibiotics'])):
    if i == ab_class['Antibiotics'][j] :
       antibio_sep_col['drug'].replace(i,ab_class['Class'][j],inplace=True)
    else : pass

convert:  99%|█████████▉| 39630/40001 [00:15<00:00, 2584.64it/s]


In [145]:
antibio_sep_col['drug'].value_counts()

glycopeptides                      11967
cephalosporins                     11005
penicillins                         6127
fluoroquinolones                    3684
nitroimidazole                      3001
macrolides                          1927
sulfonamides(diaminopyrimidine)      666
licosamides(lincomycin)              442
aminoglycosides                      324
tetracyclines                        230
monobactams                          145
nitrofuran(toin)                      76
rifamycins                            36
Name: drug, dtype: int64

In [146]:
antibio=antibio_sep_col[['hadm_id', 'stay_id','drug','culture_time','sofa_score','liver','renal']]

In [147]:
antibio

,hadm_id,stay_id,drug,culture_time,sofa_score,liver,renal
0,26184834,37510196,sulfonamides(diaminopyrimidine),2131-01-10 16:36:00,3,0,0
1,26184834,37510196,glycopeptides,2131-01-10 16:36:00,3,0,0
3,23581541,39060235,cephalosporins,2160-05-18 14:40:00,2,0,0
4,23822395,33685454,macrolides,2129-08-04 17:04:00,2,0,1
5,28994087,31090461,fluoroquinolones,2130-09-23 20:30:00,3,0,2
...,...,...,...,...,...,...,...
42211,26489544,34403689,fluoroquinolones,2132-10-01 00:38:00,3,0,1
42212,21439025,37364566,fluoroquinolones,2162-08-16 00:00:00,7,3,0
42213,26785317,32336619,cephalosporins,2148-11-19 17:00:00,3,0,0
42214,25304202,31070865,fluoroquinolones,2139-10-10 16:45:00,2,0,0


In [148]:
bloodculture = microbiologyevents[microbiologyevents['spec_type_desc']=='BLOOD CULTURE']

In [149]:
bloodculture_org =bloodculture[['hadm_id','charttime','org_name','ab_name','interpretation']].dropna(subset=['hadm_id','org_name','ab_name'])
bloodculture_org.columns=['hadm_id','culture_time','org_name','ab_name','ab_interpretation']

In [150]:
bloodculture_org

,hadm_id,culture_time,org_name,ab_name,ab_interpretation
1871,25777141.0,2144-06-06 01:18:00,STAPH AUREUS COAG +,ERYTHROMYCIN,R
1872,25777141.0,2144-06-06 01:18:00,STAPH AUREUS COAG +,CLINDAMYCIN,R
1873,25777141.0,2144-06-06 01:18:00,STAPH AUREUS COAG +,TRIMETHOPRIM/SULFA,S
1874,25777141.0,2144-06-06 01:18:00,STAPH AUREUS COAG +,TETRACYCLINE,S
1875,25777141.0,2144-06-06 01:18:00,STAPH AUREUS COAG +,GENTAMICIN,S
...,...,...,...,...,...
3395038,21439025.0,2162-08-15 09:33:00,STAPH AUREUS COAG +,CLINDAMYCIN,S
3395039,21439025.0,2162-08-15 09:33:00,STAPH AUREUS COAG +,TRIMETHOPRIM/SULFA,S
3395040,21439025.0,2162-08-15 09:33:00,STAPH AUREUS COAG +,GENTAMICIN,S
3395041,21439025.0,2162-08-15 09:33:00,STAPH AUREUS COAG +,OXACILLIN,S


In [151]:
patients_micro = pd.merge(left = antibio, right = bloodculture_org, how='inner', left_on = ['hadm_id','culture_time']	, right_on=['hadm_id','culture_time'])

In [152]:
patients_micro

,hadm_id,stay_id,drug,culture_time,sofa_score,liver,renal,org_name,ab_name,ab_interpretation
0,22987108,32359580,glycopeptides,2146-06-22 20:00:00,11,2,3,ESCHERICHIA COLI,AMPICILLIN,S
1,22987108,32359580,glycopeptides,2146-06-22 20:00:00,11,2,3,ESCHERICHIA COLI,CEFAZOLIN,S
2,22987108,32359580,glycopeptides,2146-06-22 20:00:00,11,2,3,ESCHERICHIA COLI,TRIMETHOPRIM/SULFA,S
3,22987108,32359580,glycopeptides,2146-06-22 20:00:00,11,2,3,ESCHERICHIA COLI,GENTAMICIN,S
4,22987108,32359580,glycopeptides,2146-06-22 20:00:00,11,2,3,ESCHERICHIA COLI,TOBRAMYCIN,S
...,...,...,...,...,...,...,...,...,...,...
3541,28414691,33339952,glycopeptides,2140-05-13 09:55:00,4,0,1,ESCHERICHIA COLI,CIPROFLOXACIN,R
3542,28414691,33339952,glycopeptides,2140-05-13 09:55:00,4,0,1,ESCHERICHIA COLI,AMPICILLIN/SULBACTAM,R
3543,28414691,33339952,glycopeptides,2140-05-13 09:55:00,4,0,1,ESCHERICHIA COLI,PIPERACILLIN/TAZO,S
3544,28414691,33339952,glycopeptides,2140-05-13 09:55:00,4,0,1,ESCHERICHIA COLI,CEFEPIME,S


In [157]:
hami=patients_micro['hadm_id'].unique()
def pat_ex():
  x=np.random.randint(0,269)
  t=hami[x]
  ph_ex=patients_micro[patients_micro['hadm_id']==t]
  print(ph_ex)

In [158]:
pat_ex()

      hadm_id   stay_id             drug         culture_time  sofa_score  \
517  21981089  36231647  nitroimidazole   2126-05-07 21:40:00           4   
518  21981089  36231647  nitroimidazole   2126-05-07 21:40:00           4   
519  21981089  36231647  nitroimidazole   2126-05-07 21:40:00           4   
520  21981089  36231647  nitroimidazole   2126-05-07 21:40:00           4   
521  21981089  36231647  nitroimidazole   2126-05-07 21:40:00           4   
522  21981089  36231647  nitroimidazole   2126-05-07 21:40:00           4   
523  21981089  36231647  nitroimidazole   2126-05-07 21:40:00           4   
524  21981089  36231647      penicillins  2126-05-07 21:40:00           4   
525  21981089  36231647      penicillins  2126-05-07 21:40:00           4   
526  21981089  36231647      penicillins  2126-05-07 21:40:00           4   
527  21981089  36231647      penicillins  2126-05-07 21:40:00           4   
528  21981089  36231647      penicillins  2126-05-07 21:40:00           4   

In [159]:
pat_ex()

       hadm_id   stay_id           drug         culture_time  sofa_score  \
1648  29305781  38290115    penicillins  2144-07-28 20:00:00           2   
1649  29305781  38290115    penicillins  2144-07-28 20:00:00           2   
1650  29305781  38290115    penicillins  2144-07-28 20:00:00           2   
1651  29305781  38290115    penicillins  2144-07-28 20:00:00           2   
1652  29305781  38290115    penicillins  2144-07-28 20:00:00           2   
1653  29305781  38290115    penicillins  2144-07-28 20:00:00           2   
1654  29305781  38290115    penicillins  2144-07-28 20:00:00           2   
1655  29305781  38290115    penicillins  2144-07-28 20:00:00           2   
1656  29305781  38290115    penicillins  2144-07-28 20:00:00           2   
1657  29305781  38290115    penicillins  2144-07-28 20:00:00           2   
1658  29305781  38290115    penicillins  2144-07-28 20:00:00           2   
1659  29305781  38290115  glycopeptides  2144-07-28 20:00:00           2   
1660  293057

In [160]:
pat_ex()

       hadm_id   stay_id            drug         culture_time  sofa_score  \
3277  21311611  31288304   glycopeptides  2154-05-22 14:23:00           3   
3278  21311611  31288304   glycopeptides  2154-05-22 14:23:00           3   
3279  21311611  31288304   glycopeptides  2154-05-22 14:23:00           3   
3280  21311611  31288304   glycopeptides  2154-05-22 14:23:00           3   
3281  21311611  31288304   glycopeptides  2154-05-22 14:23:00           3   
3282  21311611  31288304   glycopeptides  2154-05-22 14:23:00           3   
3283  21311611  31288304  cephalosporins  2154-05-22 14:23:00           3   
3284  21311611  31288304  cephalosporins  2154-05-22 14:23:00           3   
3285  21311611  31288304  cephalosporins  2154-05-22 14:23:00           3   
3286  21311611  31288304  cephalosporins  2154-05-22 14:23:00           3   
3287  21311611  31288304  cephalosporins  2154-05-22 14:23:00           3   
3288  21311611  31288304  cephalosporins  2154-05-22 14:23:00           3   

In [161]:
pat_ex()

      hadm_id   stay_id           drug         culture_time  sofa_score  \
737  25778212  39862840  glycopeptides  2158-07-13 10:17:00           4   
738  25778212  39862840  glycopeptides  2158-07-13 10:17:00           4   
739  25778212  39862840  glycopeptides  2158-07-13 10:17:00           4   
740  25778212  39862840  glycopeptides  2158-07-13 10:17:00           4   
741  25778212  39862840  glycopeptides  2158-07-13 10:17:00           4   
742  25778212  39862840  glycopeptides  2158-07-13 10:17:00           4   

     liver  renal             org_name             ab_name ab_interpretation  
737      3      0  STAPH AUREUS COAG +        ERYTHROMYCIN                 S  
738      3      0  STAPH AUREUS COAG +         CLINDAMYCIN                 S  
739      3      0  STAPH AUREUS COAG +  TRIMETHOPRIM/SULFA                 S  
740      3      0  STAPH AUREUS COAG +          GENTAMICIN                 S  
741      3      0  STAPH AUREUS COAG +           OXACILLIN                 S  


In [162]:
pat_ex()

       hadm_id   stay_id            drug         culture_time  sofa_score  \
2034  26543049  32263856  cephalosporins  2123-04-03 13:15:00           2   
2035  26543049  32263856  cephalosporins  2123-04-03 13:15:00           2   
2036  26543049  32263856  cephalosporins  2123-04-03 13:15:00           2   
2037  26543049  32263856  cephalosporins  2123-04-03 13:15:00           2   
2038  26543049  32263856  cephalosporins  2123-04-03 13:15:00           2   
2039  26543049  32263856  cephalosporins  2123-04-03 13:15:00           2   
2040  26543049  32263856  cephalosporins  2123-04-03 13:15:00           2   
2041  26543049  32263856  cephalosporins  2123-04-03 13:15:00           2   
2042  26543049  32263856  cephalosporins  2123-04-03 13:15:00           2   
2043  26543049  32263856  cephalosporins  2123-04-03 13:15:00           2   
2044  26543049  32263856  cephalosporins  2123-04-03 13:15:00           2   
2045  26543049  32263856  cephalosporins  2123-04-03 13:15:00           2   

In [163]:
pat_ex()

      hadm_id   stay_id                     drug         culture_time  \
423  29492382  39212435              penicillins  2171-01-05 14:13:00   
424  29492382  39212435              penicillins  2171-01-05 14:13:00   
425  29492382  39212435              penicillins  2171-01-05 14:13:00   
426  29492382  39212435              penicillins  2171-01-05 14:13:00   
427  29492382  39212435              penicillins  2171-01-05 14:13:00   
428  29492382  39212435              penicillins  2171-01-05 14:13:00   
429  29492382  39212435              penicillins  2171-01-05 14:13:00   
430  29492382  39212435              penicillins  2171-01-05 14:13:00   
431  29492382  39212435              penicillins  2171-01-05 14:13:00   
432  29492382  39212435              penicillins  2171-01-05 14:13:00   
433  29492382  39212435              penicillins  2171-01-05 14:13:00   
434  29492382  39212435  licosamides(lincomycin)  2171-01-05 14:13:00   
435  29492382  39212435  licosamides(lincomycin)  2

In [164]:
pat_ex()

       hadm_id   stay_id         drug         culture_time  sofa_score  liver  \
1596  29136655  39354036  penicillins  2145-06-15 01:12:00           2      0   
1597  29136655  39354036  penicillins  2145-06-15 01:12:00           2      0   
1598  29136655  39354036  penicillins  2145-06-15 01:12:00           2      0   
1599  29136655  39354036  penicillins  2145-06-15 01:12:00           2      0   
1600  29136655  39354036  penicillins  2145-06-15 01:12:00           2      0   

      renal              org_name       ab_name ab_interpretation  
1596      0  ENTEROCOCCUS FAECIUM  PENICILLIN G                 R  
1597      0  ENTEROCOCCUS FAECIUM    AMPICILLIN                 R  
1598      0  ENTEROCOCCUS FAECIUM    VANCOMYCIN                 R  
1599      0  ENTEROCOCCUS FAECIUM    DAPTOMYCIN                 S  
1600      0  ENTEROCOCCUS FAECIUM     LINEZOLID                 S  


In [165]:
pat_ex()

       hadm_id   stay_id           drug         culture_time  sofa_score  \
1766  22764516  33288843  glycopeptides  2157-11-19 16:02:00           2   
1767  22764516  33288843  glycopeptides  2157-11-19 16:02:00           2   
1768  22764516  33288843  glycopeptides  2157-11-19 16:02:00           2   
1769  22764516  33288843  glycopeptides  2157-11-19 16:02:00           2   
1770  22764516  33288843  glycopeptides  2157-11-19 16:02:00           2   
1771  22764516  33288843  glycopeptides  2157-11-19 16:02:00           2   
1772  22764516  33288843  glycopeptides  2157-11-19 16:02:00           2   

      liver  renal                    org_name       ab_name ab_interpretation  
1766      0      0  STAPHYLOCOCCUS EPIDERMIDIS  ERYTHROMYCIN                 R  
1767      0      0  STAPHYLOCOCCUS EPIDERMIDIS   CLINDAMYCIN                 R  
1768      0      0  STAPHYLOCOCCUS EPIDERMIDIS    GENTAMICIN                 S  
1769      0      0  STAPHYLOCOCCUS EPIDERMIDIS    VANCOMYCIN       

In [166]:
pat_ex()

      hadm_id   stay_id            drug         culture_time  sofa_score  \
827  20462195  32533010  cephalosporins  2188-10-17 22:37:00           8   
828  20462195  32533010  cephalosporins  2188-10-17 22:37:00           8   
829  20462195  32533010  cephalosporins  2188-10-17 22:37:00           8   
830  20462195  32533010  cephalosporins  2188-10-17 22:37:00           8   
831  20462195  32533010  cephalosporins  2188-10-17 22:37:00           8   
832  20462195  32533010  cephalosporins  2188-10-17 22:37:00           8   
833  20462195  32533010  cephalosporins  2188-10-17 22:37:00           8   
834  20462195  32533010  cephalosporins  2188-10-17 22:37:00           8   
835  20462195  32533010  cephalosporins  2188-10-17 22:37:00           8   

     liver  renal             org_name             ab_name ab_interpretation  
827      4      0  STAPH AUREUS COAG +        PENICILLIN G                 R  
828      4      0  STAPH AUREUS COAG +        ERYTHROMYCIN                 R  
82

In [169]:
pat_ex()

      hadm_id   stay_id            drug         culture_time  sofa_score  \
377  23869046  30354158  cephalosporins  2135-03-04 00:39:00           2   
378  23869046  30354158  cephalosporins  2135-03-04 00:39:00           2   
379  23869046  30354158  cephalosporins  2135-03-04 00:39:00           2   
380  23869046  30354158  cephalosporins  2135-03-04 00:39:00           2   
381  23869046  30354158  cephalosporins  2135-03-04 00:39:00           2   
382  23869046  30354158  cephalosporins  2135-03-04 00:39:00           2   
383  23869046  30354158  cephalosporins  2135-03-04 00:39:00           2   
384  23869046  30354158  cephalosporins  2135-03-04 00:39:00           2   
385  23869046  30354158  cephalosporins  2135-03-04 00:39:00           2   
386  23869046  30354158  cephalosporins  2135-03-04 00:39:00           2   
387  23869046  30354158   glycopeptides  2135-03-04 00:39:00           2   
388  23869046  30354158   glycopeptides  2135-03-04 00:39:00           2   
389  2386904

In [170]:
pat_ex()

       hadm_id   stay_id            drug         culture_time  sofa_score  \
2087  28649799  31583513  cephalosporins  2181-02-05 18:42:00           5   
2088  28649799  31583513  cephalosporins  2181-02-05 18:42:00           5   
2089  28649799  31583513  cephalosporins  2181-02-05 18:42:00           5   
2090  28649799  31583513  cephalosporins  2181-02-05 18:42:00           5   
2091  28649799  31583513  cephalosporins  2181-02-05 18:42:00           5   
2092  28649799  31583513  cephalosporins  2181-02-05 18:42:00           5   
2093  28649799  31583513  cephalosporins  2181-02-05 18:42:00           5   
2094  28649799  31583513  cephalosporins  2181-02-05 18:42:00           5   
2095  28649799  31583513  cephalosporins  2181-02-05 18:42:00           5   
2096  28649799  31583513  cephalosporins  2181-02-05 18:42:00           5   
2097  28649799  31583513  cephalosporins  2181-02-05 18:42:00           5   
2098  28649799  31583513  cephalosporins  2181-02-05 18:42:00           5   